In [1]:
import pandas as pd
import requests

In [ ]:
# REGIONALES (candidatos 267 listas)
df_regionales = pd.read_json('regionales.json')
df_regionales.to_csv('regionales.csv')

In [ ]:
# PROVINCIALES (candidatos 1562)
df_provinciales = pd.read_json('provinciales.json')
df_provinciales.to_csv('provinciales.csv')

In [ ]:
# DISTRITALES LIMA METROPOLITANA
df_distritales = pd.read_json('distritales_lima_metropolitana.json')
df_distritales.to_csv('distritales_lima_metropolitana.csv')

In [ ]:
# DISTRITALES
df_distritales = pd.read_json('distritales.json')
df_distritales.to_csv('distritales.csv')

In [ ]:
# LISTA DE CANDIDATOS

for txCodExpedienteExt in df_distritales['txCodExpedienteExt']:
    url = f'https://apiplataformaelectoral3.jne.gob.pe/api/v1/plan-gobierno/candidatos?IdProcesoElectoral=113&TxCodExpedienteExt={txCodExpedienteExt}'
    resp = requests.get(url=url, verify=False)
    data = resp.json()['data']
    if data != []:
        df = pd.DataFrame(data)
        df['txCodExpedienteExt'] = txCodExpedienteExt
        df.to_csv(f"listas_distritales_lima_metropolitana/{txCodExpedienteExt}.csv")

In [2]:
# CONCATENAR
import pandas as pd
import glob
import os

# setting the path for joining multiple files
files = os.path.join("listas_distritales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df_candidatos = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
df_candidatos.to_csv('candidatos_distrital.csv')
df_candidatos

Resultant CSV after joining all CSV files at a particular location...


,idHojaVida,feNacimiento,txSexo,designado,idProcesoElectoral,numeroDocumento,nombreCompleto,estado,idOrganizacionPolitica,organizacionPolitica,numeroCandidato,rutaArchivo,postulaDepartamento,postulaProvincia,postulaDistrito,cargo,nativo,txCodExpedienteExt
0,165434.0,1959-08-24T00:00:00,M,NaN,0.0,1106239,ROLANDO BEDON FERRER,INSCRITO,22.0,RENOVACION POPULAR,0.0,/Assets/Fotos-HojaVida/165434.jpg,LIMA,BARRANCA,SUPE,['ALCALDE DISTRITAL'],NO,ERM.2022014697
1,165435.0,1974-04-28T00:00:00,M,NaN,0.0,15849959,VIDAL PUNCHAN QUISPE ROJAS,INSCRITO,22.0,RENOVACION POPULAR,1.0,/Assets/Fotos-HojaVida/165435.jpg,LIMA,BARRANCA,SUPE,['REGIDOR DISTRITAL'],NO,ERM.2022014697
2,165436.0,1984-07-08T00:00:00,F,NaN,0.0,43063758,BYRE CONNIE GONZALES AYALA,INSCRITO,22.0,RENOVACION POPULAR,2.0,/Assets/Fotos-HojaVida/165436.jpg,LIMA,BARRANCA,SUPE,['REGIDOR DISTRITAL'],NO,ERM.2022014697
3,165437.0,1964-08-11T00:00:00,M,NaN,0.0,15590731,JOSUE CALEB AMARO BENTO,INSCRITO,22.0,RENOVACION POPULAR,3.0,/Assets/Fotos-HojaVida/165437.jpg,LIMA,BARRANCA,SUPE,['REGIDOR DISTRITAL'],NO,ERM.2022014697
4,165438.0,1999-11-27T00:00:00,F,NaN,0.0,76098879,OLGA CAROLINA GLORIA CASTILLO,INSCRITO,22.0,RENOVACION POPULAR,4.0,/Assets/Fotos-HojaVida/165438.jpg,LIMA,BARRANCA,SUPE,['REGIDOR DISTRITAL'],NO,ERM.2022014697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66053,142629.0,1972-12-11T00:00:00,M,NaN,0.0,2850293,MANUEL SILVA YAMUNAQUE,INSCRITO,2862.0,CONTIGO REGION,1.0,/Assets/Fotos-HojaVida/142629.jpg,PIURA,PIURA,EL TALLAN,['REGIDOR DISTRITAL'],NO,ERM.2022008577
66054,142630.0,1975-04-02T00:00:00,F,NaN,0.0,2850231,MARIA DEL PILAR COVEÑAS SULLON,INSCRITO,2862.0,CONTIGO REGION,2.0,/Assets/Fotos-HojaVida/142630.jpg,PIURA,PIURA,EL TALLAN,['REGIDOR DISTRITAL'],NO,ERM.2022008577
66055,142631.0,1968-06-20T00:00:00,M,NaN,0.0,2860642,ALFONZO NEGRI ESTRADA,INSCRITO,2862.0,CONTIGO REGION,3.0,/Assets/Fotos-HojaVida/142631.jpg,PIURA,PIURA,EL TALLAN,['REGIDOR DISTRITAL'],NO,ERM.2022008577
66056,142632.0,2003-04-21T00:00:00,F,NaN,0.0,70313465,ROSA ANAHI COVEÑAS MORE,INSCRITO,2862.0,CONTIGO REGION,4.0,/Assets/Fotos-HojaVida/142632.jpg,PIURA,PIURA,EL TALLAN,['REGIDOR DISTRITAL'],NO,ERM.2022008577


In [7]:
# LIMPIEZA - verificado
df_partidos = pd.read_csv('partidos_distrital_lima_metropolitana.csv')
df_candidatos_limpio = df_candidatos[df_candidatos.txCodExpedienteExt.isin(df_partidos.txCodExpedienteExt)]
df_candidatos_limpio.to_csv('candidatos_distrital_lima_limpio.csv')

In [8]:
# ELIMINADOS - verificado
df_candidatos_eliminados = df_candidatos[~df_candidatos.txCodExpedienteExt.isin(df_partidos.txCodExpedienteExt)]
df_candidatos_eliminados.to_csv('candidatos_distrital_lima_eliminado.csv')

In [3]:
# MERGE
df1 = pd.read_csv('candidatos_distrital.csv')
df2 = pd.read_csv('partidos_distrital.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_distrital_final.csv')

/tmp/ipykernel_36090/1881693647.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('candidatos_distrital.csv')


In [19]:
# solo gobernadores y vice en regionales
df_gobernadores_vicegobernadores_regional = pd.read_csv('base_datos_regional_final.csv')
df_gobernadores_vicegobernadores_regional = df_gobernadores_vicegobernadores_regional[df_gobernadores_vicegobernadores_regional['cargo'].str.contains('GOBERNADOR REGIONAL' or 'VICEGOBERNADOR REGIONAL')].drop(columns=['Unnamed: 0'])
df_gobernadores_vicegobernadores_regional.cargo = df_gobernadores_vicegobernadores_regional.cargo.apply(lambda x: 'VICEGOBERNADOR REGIONAL' if 'VICEGOBERNADOR REGIONAL' in x else 'GOBERNADOR REGIONAL')
df_gobernadores_vicegobernadores_regional.to_csv('solo_gobernadores_y_vicegobernadores.csv', index=False)

In [3]:
# solo alcaldes provinciales
df_alcaldes_provincial = pd.read_csv('base_datos_provincial_final.csv')
df_alcaldes_provincial = df_alcaldes_provincial[df_alcaldes_provincial['cargo'].str.contains('ALCALDE PROVINCIAL')].drop(columns=['Unnamed: 0'])
df_alcaldes_provincial['cargo'] = 'ALCALDE PROVINCIAL'
df_alcaldes_provincial.to_csv('solo_alcaldes_provinciales.csv', index=False)

In [13]:
# solo alcaldes distrital Lima Metropolitana
df_alcaldes_distrital = pd.read_csv('base_datos_distrital_lima_metropolitana_final.csv')
df_alcaldes_distrital = df_alcaldes_distrital[df_alcaldes_distrital['cargo'].str.contains('ALCALDE DISTRITAL')].drop(columns=['Unnamed: 0'])
df_alcaldes_distrital['cargo'] = 'ALCALDE DISTRITAL'
df_alcaldes_distrital.to_csv('solo_alcaldes_distritales_lima_metropolitana.csv', index=False)

In [ ]:
# Numero de partidos: 147
df2 = pd.read_csv('partidos_general.csv')
numero_partidos = df2['txOrganizacionPolitica'].nunique()
numero_partidos

In [ ]:
# Numero de apellidos unicos: 107330 de 135056 DNIs
df3 = pd.read_csv('base_datos_final.csv')
df3['Apellidos'] = (df3['txNombreCompleto'].str.split().str[-2:].str.join(' '))
personas_unicas = df3['txDocumentoIdentidad'].nunique()
apellidos_unicos = df3['Apellidos'].nunique()
df3.to_csv('base_datos_final.csv')

In [ ]:
# DNI duplicados: 369 registros, 184 personas
dni_duplicados = df3[df3['txDocumentoIdentidad'].duplicated(keep=False)]
dni_duplicados.to_csv('dni_duplicados.csv')
personas_unicas = dni_duplicados['txDocumentoIdentidad'].nunique()
personas_unicas
moda = dni_duplicados['txDocumentoIdentidad'].mode()
moda

In [ ]:
# Elecciones regionales: gobernadores y vice gobernadores
df_regionales_gobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_gobernadores = df_regionales_gobernadores[df_regionales_gobernadores['txCargoEleccion']=='GOBERNADOR REGIONAL'].reset_index()
df_regionales_gobernadores.to_csv('gobernadores_regionales.csv')
df_regionales_vicegobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_vicegobernadores = df_regionales_vicegobernadores[df_regionales_vicegobernadores['txCargoEleccion']=='VICEGOBERNADOR REGIONAL'].reset_index()
df_regionales_vicegobernadores.to_csv('vicegobernadores_regionales.csv')

In [ ]:
# Elecciones provinciales: alcaldes
df_provinciales_alcaldes = pd.read_csv('base_datos_final.csv')
df_provinciales_alcaldes = df_provinciales_alcaldes[df_provinciales_alcaldes['txCargoEleccion']=='ALCALDE PROVINCIAL'].reset_index()
df_provinciales_alcaldes.to_csv('alcaldes_provinciales.csv')